In [ ]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

def create_training_samples(image_folder, annotation_folder, target_size):
    training_samples = []

    for image_file in os.listdir(image_folder):
        image_path = os.path.join(image_folder, image_file)
        annotation_path = os.path.join(annotation_folder, image_file.replace(".jpg", ".txt"))
        real_img = cv2.imread(image_path)
        img = cv2.cvtColor(real_img, cv2.COLOR_BGR2RGB)

        with open(annotation_path, 'r') as annotation_file:
            lines = annotation_file.readlines()

            for line in lines:
                data = line.strip().split()
                class_label = int(data[0])
                x_center, y_center, width, height = map(float, data[1:5])


                # Convert YOLO format to (x, y, w, h)
                x = int((x_center - width / 2) * img.shape[1])
                y = int((y_center - height / 2) * img.shape[0])
                w = int(width * img.shape[1])
                h = int(height * img.shape[0])

                # Read the image and crop the region defined by the bounding box

                # cropped_region = img[y:y+h, x:x+w]
                cropped_region = img[max(y, 0):min(y+h, img.shape[0]), max(x, 0):min(x+w, img.shape[1])]

                # Resize the cropped region to the target size
                resized_cropped_region = cv2.resize(cropped_region, target_size)

                # Normalize pixel values to [0, 1]
                normalized_cropped_region = resized_cropped_region / 255.0

                # Append the training sample along with its class label
                training_samples.append((normalized_cropped_region, class_label))

    return training_samples

image_folder = 'dataset/mvcheck'
annotation_folder = 'dataset/labels'


# xception data

xception_size = (299, 299)

xception_training_samples = create_training_samples(image_folder, annotation_folder, xception_size)

xception_X_test, xception_y_test = zip(*xception_training_samples)
xception_X_test= np.array(xception_X_test)
xception_y_test =np.array(xception_y_test)

#alexnet data
alexnet_size = (227, 227)

alexnet_training_samples = create_training_samples(image_folder, annotation_folder, alexnet_size)

alexnet_X_train, alexnet_y_train = zip(*alexnet_training_samples)


alexnet_X_train =np.array(alexnet_X_train)
alexnet_y_train =np.array(alexnet_y_train)

#resnet data
resnet_size = (224, 224)

resnet_training_samples = create_training_samples(image_folder, annotation_folder, resnet_size)

resnet_X_train, resnet_y_train = zip(*resnet_training_samples)


resnet_X_train= np.array(resnet_X_train)
resnet_y_train =np.array(resnet_y_train)



In [ ]:
alexnet = load_model('models/alexnet_16batch_30epoch.h5')
inception = load_model('models/inception_16batch_10epoch.h5')
resnet = load_model('models/resnet_16batch_30epoch.h5')
xception = load_model('models/xception_16batch_30epoch.h5')
vgg = load_model('models/vgg_16batch_30epoch.h5')

In [ ]:
# Evaluate the model on the test set
resnet_score = resnet.evaluate(resnet_X_train, resnet_y_train)
vgg_score = vgg.evaluate(resnet_X_train, resnet_y_train)
alexnet_score = alexnet.evaluate(alexnet_X_train, alexnet_y_train)
xception_score= xception.evaluate(xception_X_test,xception_y_test)
inception_score = inception.evaluate(xception_X_test, xception_y_test)


In [ ]:
resnet_pred= resnet.predict(resnet_X_train)
vgg_pred= vgg.predict(resnet_X_train)
xception_pred=xception.predict(xception_X_test)
inception_pred= inception.predict(xception_X_test)
alexnet_pred=alexnet.predict(alexnet_X_train)

modified_resnet_pred = np.where(resnet_pred < 0.5, 0, 1)
modified_vgg_pred = np.where(vgg_pred < 0.5, 0, 1)
modified_xception_pred = np.where(xception_pred < 0.5, 0, 1)
modified_inception_pred = np.where(inception_pred < 0.5, 0, 1)
modified_alexnet_pred = np.where(alexnet_pred < 0.5, 0, 1)


#majoriti prediction choice
from collections import Counter

result = []

for i in range(len(modified_alexnet_pred)):
    counts = Counter([modified_resnet_pred[i][0],modified_vgg_pred[i][0], modified_xception_pred[i][0],modified_inception_pred[i][0],modified_alexnet_pred[i][0]])
    #Counter(0,0,1,1,0)
    most_common = counts.most_common(1)[0][0]
    result.append([most_common])

In [ ]:
from sklearn.metrics import accuracy_score, mean_absolute_error, mean_squared_error, r2_score, precision_score,recall_score

# For classification
accuracy = accuracy_score(xception_y_test, result)
precision=precision_score(xception_y_test, result)
recall=recall_score(xception_y_test, result)
print(accuracy, precision, recall)

In [ ]:
from sklearn import metrics

fpr, tpr, thresholds = metrics.roc_curve(xception_y_test, result, pos_label=2)
metrics.auc(fpr, tpr)

In [ ]:
from sklearn.metrics import accuracy_score, mean_absolute_error, mean_squared_error, r2_score

# For classification
accuracy = accuracy_score(xception_y_test, result)

# For regression
mae = mean_absolute_error(xception_y_test, result)
mse = mean_squared_error(xception_y_test, result)
rmse = mean_squared_error(xception_y_test, result, squared=False)  # RMSE
r_squared = r2_score(xception_y_test, result)
print('Mean Absolute Error:',mae, "Mean Squared Error:",mse, "Root Mean Squared Error:",rmse, "R2 Score:", r_squared)

In [ ]:

from tensorflow.keras.models import load_model

# Load the trained Xception model
model_path = 'models/vgg_16batch_30epoch.h5'  # Replace with your model path
xception = load_model(model_path)

# Read the image and its annotation
image_path = 'dataset/images/IMG_20240125_151732.jpg'  # Replace with your image path
annotation_path = 'dataset/labels/IMG_20240125_151732.txt'  # Replace with your annotation path

img = cv2.imread(image_path)
original_img = img.copy()  # Make a copy of the original image for visualization

# Read annotations and perform object localization
with open(annotation_path, 'r') as annotation_file:
    lines = annotation_file.readlines()

    for line in lines:
        data = line.strip().split()

        class_label = int(data[0])
        x_center, y_center, width, height = map(float, data[1:5])

        # Convert YOLO format to (x, y, w, h)
        x = int((x_center - width / 2) * img.shape[1])
        y = int((y_center - height / 2) * img.shape[0])
        w = int(width * img.shape[1])
        h = int(height * img.shape[0])

        # Crop the region defined by the bounding box
        # cropped_region = img[y:y+h, x:x+w]
        cropped_region = img[max(y, 0):min(y+h, img.shape[0]), max(x, 0):min(x+w, img.shape[1])]

        # Resize the cropped region to the target size (227, 227)
        resized_cropped_region = cv2.resize(cropped_region, (227, 227))

        # Normalize pixel values to [0, 1]
        normalized_cropped_region = resized_cropped_region / 255.0

        # Predict the class label for the cropped region
        prediction = xception.predict(np.expand_dims(normalized_cropped_region, axis=0))

        predicted_class = int(np.round(prediction)[0][0])
        if predicted_class==0:
            pclass='Paddy'
        else:
            pclass='Weed'

        # Draw bounding box and label on the original image

        cv2.rectangle(original_img, (x, y), (x+w, y+h), (0, 255, 0), 5)
        cv2.putText(original_img, f" {pclass}", (x, y+h-50), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 255), 5)

# Display the localized objects on the original image
plt.figure(figsize=(10, 10))
plt.imshow(cv2.cvtColor(original_img, cv2.COLOR_BGR2RGB))
plt.axis('off')
plt.title('Localized Objects')
plt.show()

In [ ]:
test_samples=[]
image = cv2.imread('/content/drive/MyDrive/Paddy Field/code/dataset/images/1705483831138.jpg')
annotation_path = '/content/drive/MyDrive/Paddy Field/code/dataset/labels/1705483831138.txt'
with open(annotation_path, 'r') as annotation_file:
    lines = annotation_file.readlines()

    for line in lines:
        data = line.strip().split()
        class_label = int(data[0])
        x_center, y_center, width, height = map(float, data[1:5])


        # Convert YOLO format to (x, y, w, h)
        x = int((x_center - width / 2) * image.shape[1])
        y = int((y_center - height / 2) * image.shape[0])
        w = int(width * image.shape[1])
        h = int(height * image.shape[0])

        # Read the image and crop the region defined by the bounding box

        cropped_image = image[y:y+h, x:x+w]

        # Resize the cropped region to the target size
        resized_cropped_image = cv2.resize(cropped_image, target_size)

        # Normalize pixel values to [0, 1]
        normalized_cropped_image = resized_cropped_image / 255.0

        # Append the training sample along with its class label
        test_samples.append((normalized_cropped_image, class_label))

In [ ]:
# Load the pre-trained Xception model without the top classification layer
base_model = Xception(weights='imagenet', include_top=False)

# Add a localization head
x = base_model.output
x = GlobalAveragePooling2D()(x)

x = Dense(256, activation='relu')(x)
x = Dense(1, activation='sigmoid')(x)  # 4 output nodes for bounding box coordinates (xmin, ymin, xmax, ymax)


localization_model = Model(inputs=base_model.input, outputs=x)

In [ ]:
# Load your trained model weights
localization_model.load_weights('/content/drive/MyDrive/Paddy Field/code/xception.h5')